In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import random
import time
from sklearn.model_selection import train_test_split
import cvxpy as cp

In [ ]:
transfer_train = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment3/transfer_train-1.csv')
transfer_test = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment3/transfer_test-1.csv')
train = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment3/studentspen-train.csv')
train = train.loc[train['Digit'].isin([1,9])]

In [ ]:
x_trans_train = transfer_train[['x3','y3','x4','y4','x5','y5','x6','y6']]
y_trans_train = transfer_train[['Digit']]

In [ ]:
x_test = np.array(transfer_test[['x3','y3','x4','y4','x5','y5','x6','y6']])
y_test = np.array(transfer_test[['Digit']])

In [ ]:
x_train = train[['x3','y3','x4','y4','x5','y5','x6','y6']]
y_train = train[['Digit']]

In [ ]:
y_train.loc[y_train['Digit'] != 1, 'Digit'] = -1
x = np.array(x_train)
y = np.array(y_train)
D = x.shape[1]
w1 = cp.Variable((D,1))
b1 = cp.Variable()
c = 0.001
psi1 = cp.Variable((x.shape[0],1))
objective = cp.Minimize(cp.sum(cp.square(w1)) * 0.5 + c * cp.sum(cp.square(psi1)))
constraints = [cp.multiply(y, (x @ w1 + b1)) + psi1 >= 1, psi1 >= 0]
prob = cp.Problem(objective, constraints)
prob.solve()
hyp_status = prob.status
hyp_opt_val = prob.value
hyp_W = w1
hyp_B = b1.value

print(hyp_status)
print(hyp_opt_val)
print(hyp_W)
print(hyp_B)

del objective, constraints, prob, w1, b1, x, y, D, psi1

In [ ]:
y_trans_train.loc[y_trans_train['Digit'] != 1, 'Digit'] = -1
x = np.array(x_trans_train)
y = np.array(y_trans_train)
D = x.shape[1]
w2 = cp.Variable((D, 1))
w2.value = hyp_W.value
b2 = cp.Variable()
psi2 = cp.Variable((x.shape[0],1))
objective = cp.Minimize(cp.sum(cp.square(w2)) * 0.5 + c * cp.sum(cp.square(psi2)))
constraints = [cp.multiply(y, (x @ w2 + b2)) + psi2 >= 1, cp.sum_squares(hyp_W - w2) <= 100, psi2 >= 0]
prob1 = cp.Problem(objective, constraints)
prob1.solve()
hyp_status1 = prob1.status
hyp_opt_val1 = prob1.value
hyp_W1 = w2.value
hyp_B1 = b2.value

print(hyp_status1)
print(hyp_opt_val1)
print(hyp_W1)
print(hyp_B1)

In [ ]:
total = 0
correct = 0
for i in range(len(x_test)):
    total += 1
    pred = np.dot(np.transpose(hyp_W1),x_test[i]) + hyp_B1
    if pred < 0: 
        pred = 7
    else:
        pred = 1
    if pred == y_test[i]:
        correct += 1

print(total,correct)